# Finding similar documents with Word2Vec and Soft Cosine Measure 

Soft Cosine Measure (SCM) is a promising new tool in machine learning that allows us to submit a query and return the most relevant documents. In **part 1**, we will show how you can compute SCM between two documents using the `inner_product` method. In **part 2**, we will use `SoftCosineSimilarity` to retrieve documents most similar to a query and compare the performance against other similarity measures.

First, however, we go through the basics of what Soft Cosine Measure is.

## Soft Cosine Measure basics

Soft Cosine Measure (SCM) is a method that allows us to assess the similarity between two documents in a meaningful way, even when they have no words in common. It uses a measure of similarity between words, which can be derived [2] using [word2vec][] [3] vector embeddings of words. It has been shown to outperform many of the state-of-the-art methods in the semantic text similarity task in the context of community question answering [2].

[word2vec]: https://radimrehurek.com/gensim/models/word2vec.html

SCM is illustrated below for two very similar sentences. The sentences have no words in common, but by modeling synonymy, SCM is able to accurately measure the similarity between the two sentences. The method also uses the bag-of-words vector representation of the documents (simply put, the word's frequencies in the documents). The intution behind the method is that we compute standard cosine similarity assuming that the document vectors are expressed in a non-orthogonal basis, where the angle between two basis vectors is derived from the angle between the word2vec embeddings of the corresponding words.

![Soft Cosine Measure](soft_cosine_tutorial.png)

This method was perhaps first introduced in the article “Soft Measure and Soft Cosine Measure: Measure of Features in Vector Space Model” by Grigori Sidorov, Alexander Gelbukh, Helena Gomez-Adorno, and David Pinto ([link to PDF](http://www.scielo.org.mx/pdf/cys/v18n3/v18n3a7.pdf)).

In this tutorial, we will learn how to use Gensim's SCM functionality, which consists of the `inner_product` method for one-off computation, and the `SoftCosineSimilarity` class for corpus-based similarity queries.

> **Note**:
>
> If you use this software, please consider citing [1] and [2].
>

## Running this notebook
You can download this [Jupyter notebook](http://jupyter.org/), and run it on your own computer, provided you have installed the `gensim`, `jupyter`, `sklearn`, `pyemd`, and `wmd` Python packages.

The notebook was run on an Ubuntu machine with an Intel core i7-6700HQ CPU 3.10GHz (4 cores) and 16 GB memory. Assuming all resources required by the notebook have already been downloaded, running the entire notebook on this machine takes about 30 minutes.

In [1]:
# Initialize logging.
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Part 1: Computing the Soft Cosine Measure

To use SCM, we need some word embeddings first of all. You could train a [word2vec][] (see tutorial [here](http://rare-technologies.com/word2vec-tutorial/)) model on some corpus, but we will use pre-trained word2vec embeddings.

[word2vec]: https://radimrehurek.com/gensim/models/word2vec.html

Let's create some sentences to compare.

In [2]:
sentence_obama = 'Obama speaks to the media in Illinois'.lower().split()
sentence_president = 'The president greets the press in Chicago'.lower().split()
sentence_orange = 'Having a tough time finding an orange juice press machine?'.lower().split()

The first two sentences have very similar content, and as such the SCM should be large. Before we compute the SCM, we want to remove stopwords ("the", "to", etc.), as these do not contribute a lot to the information in the sentences.

In [3]:
# Import and download stopwords from NLTK.
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.

# Remove stopwords.
stop_words = stopwords.words('english')
sentence_obama = [w for w in sentence_obama if w not in stop_words]
sentence_president = [w for w in sentence_president if w not in stop_words]
sentence_orange = [w for w in sentence_orange if w not in stop_words]

# Prepare a dictionary and a corpus.
from gensim import corpora
documents = [sentence_obama, sentence_president, sentence_orange]
dictionary = corpora.Dictionary(documents)

# Convert the sentences into bag-of-words vectors.
sentence_obama = dictionary.doc2bow(sentence_obama)
sentence_president = dictionary.doc2bow(sentence_president)
sentence_orange = dictionary.doc2bow(sentence_orange)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/novotny/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


2018-07-13 01:15:08,869 : INFO : 'pattern' package not found; tag filters are not available for English
2018-07-13 01:15:08,872 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-07-13 01:15:08,873 : INFO : built Dictionary(14 unique tokens: ['orange', 'chicago', 'president', 'press', 'time']...) from 3 documents (total 15 corpus positions)


Now, as we mentioned earlier, we will be using some downloaded pre-trained embeddings. Note that the embeddings we have chosen here require a lot of memory. We will use the embeddings to construct a term similarity matrix that will be used by the `inner_product` method.

In [4]:
%%time
import gensim.downloader as api
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix

w2v_model = api.load("glove-wiki-gigaword-50")
similarity_index = WordEmbeddingSimilarityIndex(w2v_model)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary)

2018-07-13 01:15:09,066 : INFO : loading projection weights from /home/novotny/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
2018-07-13 01:15:37,302 : INFO : loaded (400000, 50) matrix from /home/novotny/gensim-data/glove-wiki-gigaword-50/glove-wiki-gigaword-50.gz
2018-07-13 01:15:37,303 : INFO : constructing a sparse term similarity matrix using <gensim.models.keyedvectors.WordEmbeddingSimilarityIndex object at 0x7fa915cc2cc0>
2018-07-13 01:15:37,304 : INFO : iterating over columns in dictionary order
2018-07-13 01:15:37,305 : INFO : PROGRESS: at 7.14% columns (1 / 14, 7.142857% density, 7.142857% projected density)
2018-07-13 01:15:37,306 : INFO : precomputing L2-norms of word weight vectors
/mnt/storage/home/novotny/.virtualenvs/gensim/lib/python3.4/site-packages/gensim-3.4.0-py3.4-linux-x86_64.egg/gensim/matutils.py:738: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.

CPU times: user 29.7 s, sys: 2.95 s, total: 32.7 s
Wall time: 28.7 s


Let's compute SCM using the `inner_product` method.

In [5]:
similarity = similarity_matrix.inner_product(sentence_obama, sentence_president, normalized=True)
print('similarity = %.4f' % similarity)

similarity = 0.3790


Let's try the same thing with two completely unrelated sentences. Notice that the similarity is smaller.

In [6]:
similarity = similarity_matrix.inner_product(sentence_obama, sentence_orange, normalized=True)
print('similarity = %.4f' % similarity)

similarity = 0.1108


## Part 2: Similarity queries using `SoftCosineSimilarity`
You can use SCM to get the most similar documents to a query, using the `SoftCosineSimilarity` class. Its interface is similar to what is described in the [Similarity Queries](https://radimrehurek.com/gensim/tut3.html) Gensim tutorial.

### Qatar Living unannotated dataset
Contestants solving the community question answering task in the [SemEval 2016][semeval16] and [2017][semeval17] competitions had an unannotated dataset of 189,941 questions and 1,894,456 comments from the [Qatar Living][ql] discussion forums. As our first step, we will use the same dataset to build a corpus.

[semeval16]: http://alt.qcri.org/semeval2016/task3/
[semeval17]: http://alt.qcri.org/semeval2017/task3/
[ql]: http://www.qatarliving.com/forum

In [7]:
%%time
from itertools import chain
import json
from re import sub
from os.path import isfile

import gensim.downloader as api
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk import download


download("stopwords")  # Download stopwords list.
stopwords = set(stopwords.words("english"))

def preprocess(doc):
    doc = sub(r'<img[^<>]+(>|$)', " image_token ", doc)
    doc = sub(r'<[^<>]+(>|$)', " ", doc)
    doc = sub(r'\[img_assist[^]]*?\]', " ", doc)
    doc = sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', " url_token ", doc)
    return [token for token in simple_preprocess(doc, min_len=0, max_len=float("inf")) if token not in stopwords]

corpus = list(chain(*[
    chain(
        [preprocess(thread["RelQuestion"]["RelQSubject"]), preprocess(thread["RelQuestion"]["RelQBody"])],
        [preprocess(relcomment["RelCText"]) for relcomment in thread["RelComments"]])
    for thread in api.load("semeval-2016-2017-task3-subtaskA-unannotated")]))

print("Number of documents: %d" % len(documents))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/novotny/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Number of documents: 3
CPU times: user 2min 38s, sys: 1.71 s, total: 2min 40s
Wall time: 2min 40s


Using the corpus we have just build, we will now construct a [dictionary][], a [TF-IDF model][tfidf], a [word2vec model][word2vec], and a term similarity matrix.

[dictionary]: https://radimrehurek.com/gensim/corpora/dictionary.html
[tfidf]: https://radimrehurek.com/gensim/models/tfidfmodel.html
[word2vec]: https://radimrehurek.com/gensim/models/word2vec.html

In [8]:
%%time
from multiprocessing import cpu_count

from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models import Word2Vec
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SparseTermSimilarityMatrix

dictionary = Dictionary(corpus)
tfidf = TfidfModel(dictionary=dictionary)
w2v_model = Word2Vec(corpus, workers=cpu_count(), min_count=5, size=300, seed=12345)
similarity_index = WordEmbeddingSimilarityIndex(w2v_model.wv)
similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary, tfidf, nonzero_limit=100)

2018-07-13 01:18:18,526 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-07-13 01:19:19,198 : INFO : adding document #2270000 to Dictionary(462407 unique tokens: ['assymmetric', 'oronamin', 'govn', 'بلد', 'moussaoui']...)
2018-07-13 01:19:19,310 : INFO : built Dictionary(462807 unique tokens: ['assymmetric', 'oronamin', 'govn', 'بلد', 'moussaoui']...) from 2274338 documents (total 40096354 corpus positions)
2018-07-13 01:19:20,952 : INFO : collecting all words and their counts
2018-07-13 01:19:20,953 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-07-13 01:19:29,140 : INFO : PROGRESS: at sentence #2270000, processed 40024958 words, keeping 462407 word types
2018-07-13 01:19:29,155 : INFO : collected 462807 word types from a corpus of 40096354 raw words and 2274338 sentences
2018-07-13 01:19:29,156 : INFO : Loading a fresh vocabulary
2018-07-13 01:19:29,589 : INFO : effective_min_count=5 retains 104360 unique words (22% of original 4628

CPU times: user 3h 30min 24s, sys: 3h 53min 52s, total: 7h 24min 17s
Wall time: 17min 30s


### Evaluation
Next, we will load the validation and test datasets that were used by the SemEval 2016 and 2017 contestants. The datasets contain 208 original questions posted by the forum members. For each question, there is a list of 10 threads with a human annotation denoting whether or not the thread is relevant to the original question. Our task will be to order the threads so that relevant threads rank above irrelevant threads.

In [9]:
datasets = api.load("semeval-2016-2017-task3-subtaskBC")

Finally, we will perform an evaluation to compare three unsupervised similarity measures – the Soft Cosine Measure, two different implementations of the [Word Mover's Distance][wmd], and standard cosine similarity. We will use the [Mean Average Precision (MAP)][map] as an evaluation measure and 10-fold cross-validation to get an estimate of the variance of MAP for each similarity measure.

[wmd]: http://vene.ro/blog/word-movers-distance-in-python.html
[map]: https://medium.com/@pds.bangalore/mean-average-precision-abd77d0b9a7e

In [10]:
from math import isnan
from time import time

from gensim.similarities import MatrixSimilarity, WmdSimilarity, SoftCosineSimilarity
import numpy as np
from sklearn.model_selection import KFold
from wmd import WMD

def produce_test_data(dataset):
    for orgquestion in datasets[dataset]:
        query = preprocess(orgquestion["OrgQSubject"]) + preprocess(orgquestion["OrgQBody"])
        documents = [
            preprocess(thread["RelQuestion"]["RelQSubject"]) + preprocess(thread["RelQuestion"]["RelQBody"])
            for thread in orgquestion["Threads"]]
        relevance = [
            thread["RelQuestion"]["RELQ_RELEVANCE2ORGQ"] in ("PerfectMatch", "Relevant")
            for thread in orgquestion["Threads"]]
        yield query, documents, relevance

def cossim(query, documents):
    # Compute cosine similarity between the query and the documents.
    query = tfidf[dictionary.doc2bow(query)]
    index = MatrixSimilarity(
        tfidf[[dictionary.doc2bow(document) for document in documents]],
        num_features=len(dictionary))
    similarities = index[query]
    return similarities

def softcossim(query, documents):
    # Compute Soft Cosine Measure between the query and the documents.
    query = tfidf[dictionary.doc2bow(query)]
    index = SoftCosineSimilarity(
        tfidf[[dictionary.doc2bow(document) for document in documents]],
        similarity_matrix)
    similarities = index[query]
    return similarities

def wmd_gensim(query, documents):
    # Compute Word Mover's Distance as implemented in PyEMD by William Mayner
    # between the query and the documents.
    index = WmdSimilarity(documents, w2v_model)
    similarities = index[query]
    return similarities

def wmd_relax(query, documents):
    # Compute Word Mover's Distance as implemented in WMD by Source{d}
    # between the query and the documents.
    words = [word for word in set(chain(query, *documents)) if word in w2v_model.wv]
    indices, words = zip(*sorted((
        (index, word) for (index, _), word in zip(dictionary.doc2bow(words), words))))
    query = dict(tfidf[dictionary.doc2bow(query)])
    query = [
        (new_index, query[dict_index])
        for new_index, dict_index in enumerate(indices)
        if dict_index in query]
    documents = [dict(tfidf[dictionary.doc2bow(document)]) for document in documents]
    documents = [[
        (new_index, document[dict_index])
        for new_index, dict_index in enumerate(indices)
        if dict_index in document] for document in documents]
    embeddings = np.array([w2v_model.wv[word] for word in words], dtype=np.float32)
    nbow = dict(((index, list(chain([None], zip(*document)))) for index, document in enumerate(documents)))
    nbow["query"] = tuple([None] + list(zip(*query)))
    distances = WMD(embeddings, nbow, vocabulary_min=1).nearest_neighbors("query")
    similarities = [-distance for _, distance in sorted(distances)]
    return similarities

strategies = {
    "cossim" : cossim,
    "softcossim": softcossim,
    "wmd-gensim": wmd_gensim,
    "wmd-relax": wmd_relax}

def evaluate(split, strategy):
    # Perform a single round of evaluation.
    results = []
    start_time = time()
    for query, documents, relevance in split:
        similarities = strategies[strategy](query, documents)
        assert len(similarities) == len(documents)
        precision = [
            (num_correct + 1) / (num_total + 1) for num_correct, num_total in enumerate(
                num_total for num_total, (_, relevant) in enumerate(
                    sorted(zip(similarities, relevance), reverse=True)) if relevant)]
        average_precision = np.mean(precision) if precision else 0.0
        results.append(average_precision)
    return (np.mean(results) * 100, time() - start_time)

def crossvalidate(args):
    # Perform a cross-validation.
    dataset, strategy = args
    test_data = np.array(list(produce_test_data(dataset)))
    kf = KFold(n_splits=10)
    samples = []
    for _, test_index in kf.split(test_data):
        samples.append(evaluate(test_data[test_index], strategy))
    return (np.mean(samples, axis=0), np.std(samples, axis=0))

In [11]:
%%time
from multiprocessing import Pool

args_list = [
    (dataset, technique)
    for dataset in ("2016-test", "2017-test")
    for technique in ("softcossim", "wmd-gensim", "wmd-relax", "cossim")]
with Pool() as pool:
    results = pool.map(crossvalidate, args_list)

CPU times: user 2.02 s, sys: 5.2 s, total: 7.22 s
Wall time: 2min 38s


The table below shows the pointwise estimates of means and standard variances for MAP scores and elapsed times. Baselines and winners for each year are displayed in bold. We can see that the Soft Cosine Measure gives a strong performance on both the 2016 and the 2017 dataset.

In [12]:
from IPython.display import display, Markdown

output = []
baselines = [
    (("2016-test", "**Winner (UH-PRHLT-primary)**"), ((76.70, 0), (0, 0))),
    (("2016-test", "**Baseline 1 (IR)**"), ((74.75, 0), (0, 0))),
    (("2016-test", "**Baseline 2 (random)**"), ((46.98, 0), (0, 0))),
    (("2017-test", "**Winner (SimBow-primary)**"), ((47.22, 0), (0, 0))),
    (("2017-test", "**Baseline 1 (IR)**"), ((41.85, 0), (0, 0))),
    (("2017-test", "**Baseline 2 (random)**"), ((29.81, 0), (0, 0)))]
table_header = ["Dataset | Strategy | MAP score | Elapsed time (sec)", ":---|:---|:---|---:"]
for row, ((dataset, technique), ((mean_map_score, mean_duration), (std_map_score, std_duration))) \
        in enumerate(sorted(chain(zip(args_list, results), baselines), key=lambda x: (x[0][0], -x[1][0][0]))):
    if row % (len(strategies) + 3) == 0:
        output.extend(chain(["\n"], table_header))
    map_score = "%.02f ±%.02f" % (mean_map_score, std_map_score)
    duration = "%.02f ±%.02f" % (mean_duration, std_duration) if mean_duration else ""
    output.append("%s|%s|%s|%s" % (dataset, technique, map_score, duration))

display(Markdown('\n'.join(output)))



Dataset | Strategy | MAP score | Elapsed time (sec)
:---|:---|:---|---:
2016-test|softcossim|78.57 ±10.86|3.91 ±0.51
2016-test|**Winner (UH-PRHLT-primary)**|76.70 ±0.00|
2016-test|cossim|76.45 ±10.40|0.22 ±0.04
2016-test|wmd-gensim|76.04 ±11.54|12.45 ±1.29
2016-test|**Baseline 1 (IR)**|74.75 ±0.00|
2016-test|wmd-relax|73.68 ±12.69|0.33 ±0.08
2016-test|**Baseline 2 (random)**|46.98 ±0.00|


Dataset | Strategy | MAP score | Elapsed time (sec)
:---|:---|:---|---:
2017-test|**Winner (SimBow-primary)**|47.22 ±0.00|
2017-test|wmd-relax|44.83 ±16.54|0.38 ±0.08
2017-test|softcossim|44.44 ±16.07|5.17 ±0.83
2017-test|cossim|44.38 ±14.71|0.27 ±0.04
2017-test|wmd-gensim|43.85 ±15.53|15.36 ±1.92
2017-test|**Baseline 1 (IR)**|41.85 ±0.00|
2017-test|**Baseline 2 (random)**|29.81 ±0.00|

## References

1. Grigori Sidorov et al. *Soft Similarity and Soft Cosine Measure: Similarity of Features in Vector Space Model*, 2014. ([link to PDF](http://www.scielo.org.mx/pdf/cys/v18n3/v18n3a7.pdf))
2. Delphine Charlet and Geraldine Damnati, SimBow at SemEval-2017 Task 3: Soft-Cosine Semantic Similarity between Questions for Community Question Answering, 2017. ([link to PDF](http://www.aclweb.org/anthology/S17-2051))
3. Thomas Mikolov et al. Efficient Estimation of Word Representations in Vector Space, 2013. ([link to PDF](https://arxiv.org/pdf/1301.3781.pdf))